In [ ]:
#For Kaggle
#date=''

#For WL data
source='WL'
date='20241118'

saveImage=False

packageName='07-sklearn.ensemble.b-boosting'
classifierName='5-AdaBoostClassifier'
extraParameterName='22-SmoteOverSampling'

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import dataimport as di
import result as resultMd
import sampling as sp
reload(fd)
reload(sd)
reload(di)
reload(resultMd)
reload(sp)

print('done')

In [ ]:
# Results

estimator__max_depthFound=1
# learning too low makes the process very slow
learning_rateFound=0.1
n_estimatorsFound=850

resultMd.update_hyperparameter_config_result(packageName,classifierName,extraParameterName,estimator__max_depthFound,n_estimatorsFound)


print('done')

In [ ]:
import pandas as pd

dfTrx0 = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
predictors = fd.getPredictors(dfTrx0)
dfTrx=fd.getStandardScaledData('export'+date+'.csv',source,predictors)
dfTrx.head(5)

In [ ]:
from sklearn.model_selection import train_test_split

predictors = fd.getPredictors(dfTrx0)
TEST_SIZE = 0.20 # test size using_train_test_split
RANDOM_STATE = 0
x_train0, x_test, y_train0, y_test = train_test_split(dfTrx[predictors], dfTrx['Class'], test_size = TEST_SIZE, 
                                                        stratify= dfTrx['Class'],
                                                        random_state = RANDOM_STATE)

x_train, y_train = sp.smoteOverSampling(x_train0, y_train0)

# Hyperparameters tuning


In [ ]:
%%script false

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint

modelClf = AdaBoostClassifier(random_state=42)
dic_param={
    'estimator': [DecisionTreeClassifier()],
    'estimator__max_depth':randint(5,7),
    'n_estimators': randint(45,60),
    'learning_rate': [0.1]
}

res=fd.hyperparameterSelectionRandomizedSearchCVSampling(modelClf, dic_param, 'f1', x_train, y_train,iter=3)


print(res)



In [ ]:
%%script false
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

modelClf = AdaBoostClassifier(random_state=42)

 
dic_param={
     'estimator': [DecisionTreeClassifier()],
    'estimator__max_depth':[1],
    'n_estimators':[950,1000],
    'learning_rate':[0.1]
}

res=fd.hyperparameterSelectionGridSearchCVSampling(modelClf, dic_param, 'f1', x_train, y_train)
print(res)

#{'estimator': DecisionTreeClassifier(), 'estimator__max_depth': 1, 'learning_rate': 0.1, 'n_estimators': 300}
#0.18143577468756722
#scoref1 0.19466327827191868

#{'estimator': DecisionTreeClassifier(), 'estimator__max_depth': 1, 'learning_rate': 0.1, 'n_estimators': 400}
#0.20356590745376701
#scoref1 0.2236118059029515

#{'estimator': DecisionTreeClassifier(), 'estimator__max_depth': 1, 'learning_rate': 0.1, 'n_estimators': 500}
#0.20648242324167213
#scoref1 0.21861741951584726

#{'estimator': DecisionTreeClassifier(), 'estimator__max_depth': 1, 'learning_rate': 0.1, 'n_estimators': 600}
#0.23999649738454054
#scoref1 0.24828263002944062

#{'estimator': DecisionTreeClassifier(), 'estimator__max_depth': 1, 'learning_rate': 0.1, 'n_estimators': 700}
#0.24557102536085562
#scoref1 0.2644989080320311

#{'estimator': DecisionTreeClassifier(), 'estimator__max_depth': 1, 'learning_rate': 0.1, 'n_estimators': 750}
#0.2452300947649264
#scoref1 0.26034063260340634

#{'estimator': DecisionTreeClassifier(), 'estimator__max_depth': 1, 'learning_rate': 0.1, 'n_estimators': 800}
#0.2573530014752987
#scoref1 0.2770997846374731

#{'estimator': DecisionTreeClassifier(), 'estimator__max_depth': 1, 'learning_rate': 0.1, 'n_estimators': 850}
#0.2535756159534257
#scoref1 0.28296901273120345

#    'n_estimators':[850,900],
#{'estimator': DecisionTreeClassifier(), 'estimator__max_depth': 1, 'learning_rate': 0.1, 'n_estimators': 900}
#0.27484178493266553
#scoref1 0.3009685802031656

#{'estimator': DecisionTreeClassifier(), 'estimator__max_depth': 1, 'learning_rate': 0.1, 'n_estimators': 950}
#0.2783865149619385
#scoref1 0.30151946818613484


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from datetime import datetime

then= datetime.now()



modelClf = AdaBoostClassifier(random_state=42)
parameters={'learning_rate': learning_rateFound, 'n_estimators':n_estimatorsFound, 'estimator':DecisionTreeClassifier(),
           'estimator__max_depth':estimator__max_depthFound}
modelClf.set_params(**parameters)

modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

f1Train=fd.print_scores(y_train, predsTrain,'f1', False)
f1Test=fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)
fd.show_prediction_graph(modelClf, x_test,y_test)

diffF1=f1Train-f1Test
print("diffF1",diffF1)

tmp = pd.DataFrame({'Feature': predictors, 'Feature importance': modelClf.feature_importances_})
val=tmp[(tmp['Feature']=='amountBin')]['Feature importance']
amountImportance =val.values[0]
print("amount importance",amountImportance)


In [ ]:
files = fd.getAllFiles()
predictors = fd.getPredictors(dfTrx0)

range = []
f1s = []
rocs = []
loop =0
for file in files:
    loop=loop+1
    range.append(loop)
    print(file)
  
    dfTrx=fd.getStandardScaledData(file,source,predictors)
    preds = modelClf.predict(dfTrx[predictors])

    f1,mcc,roc= fd.print_scores(dfTrx['Class'], preds,'All', True)
    #fd.show_importance(modelClf,predictors)
    fd.show_confusion_matrix(dfTrx['Class'], preds)
    f1s.append(f1)
    rocs.append(roc)

fd.plt_train_test(range, f1s)
resultMd.update_performance_nextdays_result(packageName,classifierName,extraParameterName, f1s[0],f1s[1],f1s[2],f1s[3],rocs[0],rocs[1],rocs[2],rocs[3],diffF1,amountImportance)

